# Table des matières
- [À propos de Guidance](../../../../code/01.Introduce)
- [Configuration](../../../../code/01.Introduce)
- [Génération non contrainte](../../../../code/01.Introduce)
- [Parler pour Phi 3](../../../../code/01.Introduce)
- [Regex](../../../../code/01.Introduce)
- [Sélection](../../../../code/01.Introduce)
- [Raisonnement en chaîne](../../../../code/01.Introduce)
- [Génération de JSON](../../../../code/01.Introduce)
- [Génération de HTML](../../../../code/01.Introduce)


# À propos de Guidance
Guidance est une bibliothèque Python open-source éprouvée pour contrôler les sorties de tout modèle de langage (LM). Avec un seul appel d'API, vous pouvez exprimer (en Python) les contraintes programmatiques précises que le modèle doit respecter et générer une sortie structurée en JSON, Python, HTML, SQL ou tout autre format requis par le cas d'utilisation.

Guidance se distingue des techniques de sollicitation conventionnelles. Elle impose des contraintes en guidant le modèle token par token dans la couche d'inférence, produisant des sorties de meilleure qualité et réduisant les coûts et la latence de 30 à 50 % dans des scénarios hautement structurés.

Pour en savoir plus sur Guidance, visitez le [répertoire public sur GitHub](https://github.com/guidance-ai/guidance) ou regardez la [session Guidance Breakout](https://www.youtube.com/watch?v=qXMNPVVlCMs) lors de Microsoft Build.


# Configuration
1. Installez Guidance avec `pip install guidance --pre`
2. Déployez un point de terminaison Phi 3.5 mini dans Azure en vous rendant sur https://ai.azure.com/explore/models/Phi-3.5-mini-instruct/version/2/registry/azureml et en cliquant sur le bouton "Deploy"
3. Stockez la clé API de votre point de terminaison dans une variable d'environnement appelée `AZURE_PHI3_KEY` et l'URL dans une variable d'environnement appelée `AZURE_PHI3_URL`


In [ ]:
from guidance import gen, select, regex, user, assistant, system, json
from guidance.models import AzureGuidance
from json import loads as load_json_str
import os

phi3_url = os.getenv("AZURE_PHI3_URL")
phi3_api_key = os.getenv("AZURE_PHI3_KEY")
phi3_lm = AzureGuidance(f"{phi3_url}/guidance#auth={phi3_api_key}")

# Or, load from HuggingFace to run locally
# from guidance.models import Transformers
# phi3_lm = Transformers("microsoft/Phi-3-mini-4k-instruct")

# Génération sans contrainte
Le texte peut être généré sans aucune contrainte en utilisant la fonction `gen()`. Cela revient à utiliser le modèle sans Guidance.

## Formatage des conversations
Comme beaucoup de modèles de chat, Phi-3 attend que les messages entre un utilisateur et un assistant soient dans un format spécifique. Guidance prend en charge le modèle de chat de Phi-3 et gère le formatage des conversations pour vous. Pour créer des échanges, placez chaque partie de la conversation dans un bloc `with user()` ou `with assistant()`. Un bloc `with system()` peut être utilisé pour définir le message système.


In [22]:
lm = phi3_lm
with system():
    lm += "You are a helpful assistant. You have a cranky yet entertaining temperament."
with user():
    lm += "What is the capital of Australia?"
with assistant():
    lm += gen(temperature=0.8, max_tokens=100)

## Économies de jetons
Dans des scénarios très structurés, Guidance peut ignorer certains jetons et ne générer que les jetons nécessaires, améliorant ainsi les performances, augmentant l'efficacité et réduisant les coûts liés à l'API. Les jetons générés sont affichés dans ce notebook avec un arrière-plan mis en évidence. Les jetons forcés sont affichés sans mise en évidence et coûtent le même prix que les jetons d'entrée, estimés à un tiers du coût des jetons de sortie.

*Remarque :* Le premier exemple avec une génération non contrainte n'a pas pu forcer de jetons car nous n'avons fourni aucune contrainte.


# Parler pour Phi 3  
Avec Guidance, vous pouvez facilement intégrer du texte dans les réponses du modèle. Cela peut être utile si vous souhaitez orienter la sortie du modèle dans une direction spécifique.


In [5]:
lm = phi3_lm
with user():
    lm += "What is the capital of Australia?"
with assistant():
    lm += "The capital of Australia is " + gen(temperature=0.8, max_tokens=50)

# Contraindre avec des expressions régulières
Dans l'exemple précédent, Phi 3 a répondu avec des explications supplémentaires après avoir donné la réponse `Canberra`. Pour limiter la sortie du modèle à un seul mot, une expression régulière peut être utilisée.


In [6]:
lm = phi3_lm
with user():
    lm += "What is the capital of Australia?"
with assistant():
    lm += "The capital of Australia is " + regex("[A-Z][a-z]+")

# Sélectionner parmi plusieurs choix
Lorsque certaines options possibles sont connues, vous pouvez utiliser la fonction `select()` pour permettre au modèle de choisir parmi une liste d'options.


In [23]:
lm = phi3_lm
with user():
    lm += "What is the capital of Australia?"
with assistant():
    lm += "The capital of Australia is " + select(["Washington", "Canberra", "Sydney", "Melbourne"])

Avec `select()`, seul le jeton `Can` a été généré. Étant donné que `Canberra` est la seule option pouvant compléter la réponse, les jetons restants ont été forcés.


# Chaîne de Raisonnement
La chaîne de raisonnement est une technique qui peut améliorer la qualité des réponses du modèle en l'encourageant à traiter un problème étape par étape. En général, pour arriver à une réponse finale, plusieurs étapes de sollicitation sont nécessaires. Tout d'abord, demandez au modèle de réfléchir étape par étape. Ensuite, sollicitez à nouveau le modèle pour qu'il fournisse la réponse finale. Avec les API d'inférence standard pour les conversations, cela nécessite 2 appels API, et la "chaîne de raisonnement" générée par le modèle est facturée deux fois – une fois en tant que jetons de sortie lorsque le modèle la génère, et une autre fois en tant que jetons d'entrée pour le deuxième appel. Avec Guidance, tout le processus en plusieurs étapes est traité et facturé dans le cadre d'un seul appel API, réduisant ainsi les coûts et la latence.


In [8]:
gsm8k_question = "Mark has a garden with flowers. He planted plants of three different colors in it. Ten of them are yellow, and there are 80% more of those in purple. There are only 25% as many green flowers as there are yellow and purple flowers. How many flowers does Mark have in his garden?"
lm = phi3_lm
with user():
    lm += gsm8k_question
with assistant():
    lm += "Let's think step by step. " + gen(temperature=0.8, max_tokens=500)
    # Prompt for the final answer, which should be a number. Store the output in an "answer" variable.
    lm += "\nTherefore, the final answer is: " + regex(r"\d+", name="answer")

print(f"Final answer: {lm['answer']}")

Final answer: 35


# Génération de JSON
Guidance peut être utilisé pour garantir la génération de JSON conforme à un schéma JSON ou à un modèle pydantic, comme le schéma de profil utilisateur présenté ici.


In [16]:
user_json_schema = load_json_str("""{
  "$schema": "http://json-schema.org/draft-07/schema#",
  "title": "User Profile",
  "type": "object",
  "properties": {
    "username": {
      "type": "string"
    },
    "age": {
      "type": "integer"
    },
    "email": {
      "type": "string"
    }
  },
  "additionalProperties": false
}
""")

lm = phi3_lm
with user():
    lm += "Generate a JSON object for a user profile. The profile should include a username, age, email, and nothing more."

with assistant():
    lm += json(schema=user_json_schema, temperature=1.0)

In [19]:
from pydantic import BaseModel

class UserProfile(BaseModel):
    username: str
    age: int
    email: str


lm = phi3_lm
with user():
    lm += "Generate a JSON object for a user profile. The profile should include a username, age, email, and nothing more."

with assistant():
    lm += json(schema=UserProfile, temperature=1.0)

## Génération HTML

Guidance peut également être utilisé pour générer du code et respecter les exigences syntaxiques du langage de programmation. Dans cette section, nous allons créer un petit programme Guidance pour rédiger des pages web HTML très simples.

Nous allons diviser la page web en sections plus petites, chacune avec sa propre fonction Guidance. Ces sections seront ensuite combinées dans notre fonction finale pour créer une page web HTML.  
Nous exécuterons ensuite cette fonction avec un modèle compatible Guidance dans Azure AI.

*Remarque :* Il ne s'agit pas d'un générateur HTML complet ; l'objectif est de montrer comment vous pouvez créer une sortie structurée adaptée à vos besoins spécifiques.

Nous commençons par importer ce dont nous avons besoin depuis Guidance :


In [ ]:
from guidance import guidance
from guidance.library import (
    zero_or_more,
    any_char_but,
    select,
    capture,
    with_temperature,
)
from guidance.models import Model

Les pages web HTML sont très structurées, et nous allons 'forcer' ces parties de la page en utilisant Guidance.  
Lorsque nous demandons explicitement du texte au modèle, nous devons nous assurer qu'il n'inclut rien qui pourrait être interprété comme une balise - c'est-à-dire que nous devons exclure les caractères '<' et '>'.


In [ ]:
@guidance(stateless=True)
def _gen_text(lm: Model):
    return lm + zero_or_more(any_char_but(["<", ">"]))

Nous pouvons ensuite utiliser cette fonction pour générer du texte dans une balise HTML arbitraire :


In [ ]:
@guidance(stateless=True)
def _gen_text_in_tag(lm: Model, tag: str):
    lm += f"<{tag}>"
    lm += _gen_text()
    lm += f"</{tag}>"
    return lm

Maintenant, créons l'en-tête de la page.  
Dans ce cadre, nous devons générer un titre de page :


In [ ]:
@guidance(stateless=True)
def _gen_header(lm: Model):
    lm += "<head>\n"
    lm += _gen_text_in_tag("title") + "\n"
    lm += "</head>\n"
    return lm

Le corps de la page HTML sera rempli de titres et de paragraphes.  
Nous pouvons définir une fonction pour chacun :


In [ ]:
@guidance(stateless=True)
def _gen_heading(lm: Model):
    lm += select(
        options=[_gen_text_in_tag("h1"), _gen_text_in_tag("h2"), _gen_text_in_tag("h3")]
    )
    lm += "\n"
    return lm

@guidance(stateless=True)
def _gen_para(lm: Model):
    lm += _gen_text_in_tag("p")
    lm += "\n"
    return lm

Maintenant, la fonction pour définir le corps du HTML lui-même.  
Cela utilise `select()` avec `recurse=True` pour générer plusieurs titres et paragraphes :


In [ ]:
@guidance(stateless=True)
def _gen_body(lm: Model):
    lm += "<body>\n"
    lm += select(options=[_gen_heading(), _gen_para()], recurse=True)
    lm += "</body>\n"
    return lm

Ensuite, nous arrivons à la fonction qui génère la page HTML complète.  
Nous ajoutons la balise de début HTML, puis nous générons l'en-tête, ensuite le corps, et enfin nous ajoutons la balise de fin HTML :


In [ ]:
@guidance(stateless=True)
def _gen_html(lm: Model):
    lm += "<html>\n"
    lm += _gen_header()
    lm += _gen_body()
    lm += "</html>\n"
    return lm

Nous fournissons une interface conviviale, qui nous permettra de :
- Définir la température de la génération
- Capturer la page générée à partir de l'objet Model


In [ ]:
@guidance(stateless=True)
def html(
    lm,
    name: str | None = None,
    *,
    temperature: float = 0.0,
):
    return lm + capture(
        with_temperature(_gen_html(), temperature=temperature),
        name=name,
    )

In [ ]:
lm = phi3_lm

lm += "Create a web page about your life story. Split your uplifting tale into multiple paragraphs with headings:\n"
lm += html(name="html_text", temperature=0.7)

Nous pouvons ensuite écrire la sortie dans un fichier :


In [ ]:
with open('./sample_page.html', 'w') as html_file:
    html_file.write(lm["html_text"])

Et [voir le résultat](../../../../code/01.Introduce/sample_page.html).



---

**Avertissement** :  
Ce document a été traduit à l'aide du service de traduction automatique [Co-op Translator](https://github.com/Azure/co-op-translator). Bien que nous nous efforcions d'assurer l'exactitude, veuillez noter que les traductions automatisées peuvent contenir des erreurs ou des inexactitudes. Le document original dans sa langue d'origine doit être considéré comme la source faisant autorité. Pour des informations critiques, il est recommandé de recourir à une traduction professionnelle réalisée par un humain. Nous déclinons toute responsabilité en cas de malentendus ou d'interprétations erronées résultant de l'utilisation de cette traduction.
